In [1]:
# install.packages("magrittr")
# install.packages("purrr")
# install.packages("glue")
# install.packages("stringr")
library(tidyverse)
library(magrittr) # better handling of pipes
library(purrr) # to work with lists and map functions
library(glue) # to paste strings
library(stringr) # to hand strings
library(rvest) # rvest makes scraping easier
library(polite) # polite is the "polite" version of rvest
library(xml2) # makes it easier to work with HTML and XML from R

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: 'magrittr'


The following object is masked from 'package:purrr':

    set_names


The following object is masked from 'package:tidyr':

    extract



Attaching package: 'rvest'


The following object is masked from 'package:readr':

    guess_encoding




In [6]:
url = "https://www.audioculture.co.nz/music_index?category=Person"
page = read_html(url)
profile = page %>% html_nodes(".skippy-col-link") %>% html_text()
link = page %>% html_nodes(".skippy-col-link") %>% html_attr('href')
tibble(profile, link)

profile[0]

profile,link
<chr>,<chr>
"1860 Band, The",/profile/the-1860-band
25 Cents,/profile/25-cents
3 The Hard Way,/profile/3-the-hard-way
"3Ds, The",/profile/the-3ds
4 Corners,/profile/4-corners
8 Foot Sativa,/profile/8-foot-sativa
?Fog,/profile/fog
Aaradhna,/profile/aaradhna
Aaron Tokona,/profile/aaron-tokona
